In [1]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
from datetime import datetime
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter
from collections import deque
import gymnasium as gym
from gymnasium import spaces
from gymnasium.utils.env_checker import check_env
import networkx as nx
import matplotlib.pyplot as plt
import random
import collections 

class Topology():
    def __init__(self, n, model="random", density=1):
        self.n = n
        self.model = model
        self.density = density
        self.adjacency_matrix = self.make_adjacency_matrix()
        
    def make_adjacency_matrix(self) -> np.ndarray:
        if self.density < 0 or self.density > 1:
            raise ValueError("Density must be between 0 and 1.")

        n_edges = int(self.n * (self.n - 1) / 2 * self.density)
        adjacency_matrix = np.zeros((self.n, self.n))

        if self.model == "dumbbell":
            adjacency_matrix[0, self.n-1] = 1
            adjacency_matrix[self.n-1, 0] = 1
            for i in range(1, self.n//2):
                adjacency_matrix[0, i] = 1
                adjacency_matrix[i, 0] = 1
            for i in range(self.n//2+1, self.n):
                adjacency_matrix[i-1, self.n-1] = 1
                adjacency_matrix[self.n-1, i-1] = 1
        elif self.model == "linear":
            for i in range(1, self.n):
                adjacency_matrix[i-1, i] = 1
                adjacency_matrix[i, i-1] = 1
        elif self.model == "random":
            for i in range(1, self.n):
                adjacency_matrix[i-1, i] = 1
                adjacency_matrix[i, i-1] = 1
                n_edges -= 1
            if n_edges <= 0:
                return adjacency_matrix
            else:
                arr = [1]*n_edges + [0]*((self.n-1)*(self.n-2)//2 - n_edges)
                np.random.shuffle(arr)
                for i in range(0, self.n):
                    for j in range(i+2, self.n):
                        adjacency_matrix[i, j] = arr.pop()
                        adjacency_matrix[j, i] = adjacency_matrix[i, j]
        else:
            raise ValueError("Model must be dumbbell, linear, or random.")
        return adjacency_matrix

    def show_adjacency_matrix(self):
        print(self.adjacency_matrix)
        
    def get_density(self):
        return np.sum(self.adjacency_matrix) / (self.n * (self.n - 1))
    
    def save_graph_with_labels(self, path):
        rows, cols = np.where(self.adjacency_matrix == 1)
        edges = zip(rows.tolist(), cols.tolist())
        G = nx.Graph()
        G.add_edges_from(edges)
        pos = nx.kamada_kawai_layout(G)
        nx.draw_networkx(G, pos=pos, with_labels=True)
        plt.savefig(path + '/adj_graph.png')

In [2]:
# Make topology
topology = Topology(10, "dumbbell", 0.5)
topology.show_adjacency_matrix()
node_n = topology.n
N_OBSERVATIONS = 2
N_ACTIONS = 2

[[0. 1. 1. 1. 1. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 1. 1. 1. 1. 0.]]


In [3]:
import csv

class NetworkEnvironment(gym.Env):
    def __init__(self, topology, n_observations, n_actions, episode_length=300):
        super(NetworkEnvironment, self).__init__()
        self.topology = topology
        self.node_n = topology.n
        self.n_observations = n_observations
        self.n_actions = n_actions
        self.episode_length = episode_length
        self.current_step = 0
        self.aoi = np.ones(self.node_n) / episode_length
        
        self.action_space = spaces.Discrete(n_actions)
        self.observation_space = spaces.Box(low=0, high=1, shape=(self.node_n, n_observations), dtype=np.float32)
        
    def reset(self):
        self.current_step = 0
        self.aoi = np.ones(self.node_n) / self.episode_length
        return self._get_obs()
        
    def _get_obs(self):
        return np.repeat(self.aoi.reshape(-1, 1), self.n_observations, axis=1)
    
    def step(self, actions):
        transmission_probs = np.random.rand(self.node_n)
        successful_transmissions = (transmission_probs < actions).astype(int)
        
        for i in range(self.node_n):
            if successful_transmissions[i] == 1:
                neighbors = np.where(self.topology.adjacency_matrix[i] == 1)[0]
                self.aoi[neighbors] = 1 / self.episode_length
        
        self.aoi += 1 / self.episode_length
        
        self.current_step += 1
        done = self.current_step >= self.episode_length
        
        utility = np.log2(self.aoi + 1).sum()
        reward = utility
        
        return self._get_obs(), reward, done, successful_transmissions
    
    def render(self):
        pass

def calculate_average_utility_and_log(topology, n_observations, n_actions, n_episodes=100, log_file="simulation_logs.csv"):
    env = NetworkEnvironment(topology, n_observations, n_actions)
    total_utility = 0
    log_data = []

    for episode in range(n_episodes):
        obs = env.reset()
        episode_utility = 0
        
        for step in range(env.episode_length):
            actions = np.random.rand(env.node_n)
            obs, reward, done, transmissions = env.step(actions)
            episode_utility += reward

            log_entry = [episode, step] + transmissions.tolist()
            log_data.append(log_entry)
            
            if done:
                break
                
        total_utility += episode_utility
    
    # Save logs to CSV
    with open(log_file, mode='w', newline='') as file:
        writer = csv.writer(file)
        header = ["Episode", "Step"] + [f"Node_{i+1}" for i in range(env.node_n)]
        writer.writerow(header)
        writer.writerows(log_data)
    
    average_utility = total_utility / n_episodes
    return average_utility

# Example usage
topology = Topology(10, "dumbbell", 0.5)
average_utility = calculate_average_utility_and_log(topology, N_OBSERVATIONS, N_ACTIONS, n_episodes=100, log_file="simulation_logs.csv")
print(f"Average Utility over 100 episodes: {average_utility}")


Average Utility over 100 episodes: 40.05595556747705
